In this lab we will be creating 2 additional agents. Similar to how we created a stand-alone agent in 1_bedrock-single-agent lab. 

**Collaborator agent 1** – Responsible for handling general questions about mortgages.

**Collaborator agent 2** – Responsible for handling questions about existing applications.

We start by importing the libraries  

In [ ]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent
import sys

sys.path.insert(0, '..') 

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

from src.utils.bedrock_agent import AgentsForAmazonBedrock
from src.utils.secrets_helper import SecretsHelper
secrets_helper = SecretsHelper(region_name=region)

We use Anthropic Claude 3.5 Haiku model for this lab

In [ ]:
agent_foundation_model = ["anthropic.claude-3-5-haiku-20241022-v1:0"]

Get the knowledge base ID that we stored in the previous lab

In [ ]:
%store -r kb_id
%store -r knowledge_base_name
print(f"KnowledgeBase ID: {kb_id}, Name: {knowledge_base_name}")

### 1. Create the agent for general mortgage questions.

We will use the helper Agent class which provides methods for creating, configuring, and invoking individual Agents, including
associating them with Guardrails, Knowledge Bases, and Tools.

For this Agent we will also associate our Knowledge base created earlier. 

In [ ]:
general_mortgage_questions_agent_name = "general_mortgage_questions_agent"

general_mortgage_questions_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{general_mortgage_questions_agent_name}'

In [ ]:
agent_description = """Handle conversations about general mortgage questions."""

agent_instruction = """
Instructions:
You are a mortgage bot, and can answer questions about mortgage refinancing and tradeoffs of mortgage types.
Handle conversations about general mortgage questions,like high level concepts of refinincing or tradeoffs of 15-year vs 30-year terms.
Use this knowledge base to answer general questions about mortgages.

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself
6. Final response should not include your internal thought process

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

In [ ]:
agents = AgentsForAmazonBedrock()

general_mortgage_questions_agent = agents.create_agent(
    general_mortgage_questions_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model
)

general_mortgage_questions_agent_id = general_mortgage_questions_agent[0]

print("Agent created with ID: ",general_mortgage_questions_agent_id)

**Associate knowledge base**

Now that we've created the agent, let's associate the previously created knowledge base to it.

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Access this knowledge base to answer general questions about mortgages, like how to refinance, or the difference between 15-year and 30-year mortgages."""
}

In [ ]:
agents.associate_kb_with_agent(
    general_mortgage_questions_agent_id,
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

#### Test the agent

In [ ]:
response = agents.invoke(
    """Hi, What is the benefit of refinancing, if any?""", 
    general_mortgage_questions_agent_id, enable_trace=True
)
print("====================")
print(response)

create Alias 

In [ ]:
general_mortgage_questions_agent_alias_id, general_mortgage_questions_agent_alias_arn = agents.create_agent_alias(
    general_mortgage_questions_agent_id, 'general_mortgage_questions_agent_a'
)

### 2. Create the agent for existing mortgage questions.

In this section we will create an agent that answers questions on existing mortgage. In a real setting, the agent will make an API call to your mortgage application by passing in the customer id. For this lab, we will create a function but instead of making an API call, hard code the response.  


These are the steps to complete:
    
1. Create the new agent (with the helper function taking care of IAM role creation)
2. Add an action group backed by a new Lambda function (with the helper function handling IAM role creation, Lambda function creation, adding the action group to the agent, and preparing the agent)

In [ ]:
existing_mortgage_agent_name = "existing_mortgage_agent"

existing_mortgage_agent_lambda_name = "fn_existing_mortgage_agent"

existing_mortgage_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{existing_mortgage_agent_name}'

In [ ]:
agent_description = """Handle conversations about existing mortgage accounts."""

agent_instruction = """
Instructions: 
You are a mortgage bot, you greet the customer first and then you can retrieve the latest details about an existing mortgage on behalf of customers.
When starting a new session, give them a friendly greeting using their preferred name if you already have it.
never ask the user for information that you already can retrieve yourself through available actions. for example, you have actions to retrieve details about the 
existing mortgage (interest rate, balance, number of payments, mortgage maturity date, last payment date, next payment date, etc.). 
do not engage with users about topics other than an existing mortgage and greetings. 
leave those other topics for other experts to handle. 
for example, do not respond to general questions about mortgages. However, respond to the greeting by another greeting.

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself
6. Final response should not include your internal thought process


Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

In [ ]:
agents = AgentsForAmazonBedrock()

existing_mortgage_agent = agents.create_agent(
    existing_mortgage_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=True
)

existing_mortgage_agent_id = existing_mortgage_agent[0]

print("Agent created with ID: ",existing_mortgage_agent_id)

#### Create the Lambda function code
Here we create a source code file for a new Lambda function to implement the action group for our Existing Mortgage agent. Notice the **TODO** section in the code below. In this example we have hardcoded the response back from the agent but in your environment this is where your business logic would reside. 

In [ ]:
%%writefile existing_mortgage_agent_function.py
import json

def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def get_mortgage_details(customer_id):
    # TODO: Implement real business logic to retrieve mortgage status
    return {
        "account_number": customer_id,
        "outstanding_principal": 150000.0,
        "interest_rate": 4.5,
        "maturity_date": "2030-06-30",
        "payments_remaining": 72,
        "last_payment_date": "2024-06-01",
        "next_payment_due": "2024-07-01",
        "next_payment_amount": 1250.0
    }

def lambda_handler(event, context):
    print(event)
    function = event['function']
    if function == 'get_mortgage_details':
        customer_id = get_named_parameter(event, 'customer_id')
        if not customer_id:
            raise Exception("Missing mandatory parameter: customer_id")
        result = get_mortgage_details(customer_id)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Define available actions

In [ ]:
functions_def=[
        {
            "name": "get_mortgage_details",
            "description": dedent("""
Retrieves the mortgage status for a given customer ID. Returns an object containing 
details like the account number, 
outstanding principal, interest rate, maturity date, number of payments remaining, due date of next payment, 
and amount of next payment. If customer_id is not passed, function implementation
can retrieve it from session state instead."""),
            "parameters": {
                "customer_id": {
                    "description": "[optional] The unique identifier for the customer whose mortgage status is being requested.",
                    "type": "string",
                    "required": False
                }
            }
        }
    ]

Add the Lambda function and the function details as an action group for this agent and prepare it.

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=existing_mortgage_agent_name,
    lambda_function_name=existing_mortgage_agent_lambda_name,
    source_code_file="existing_mortgage_agent_function.py",
    agent_functions=functions_def,
    agent_action_group_name="existing_mortgage_actions",
    agent_action_group_description="Function to manage existing applications for a user "
)

In [ ]:
#test

response = agents.invoke(
    """Hi, I'm customer 98991. when's my next payment due?""", 
    existing_mortgage_agent_id, enable_trace=False
)
print("====================")
print(response)

Create Alias

In [ ]:
existing_mortgage_agent_alias_id, existing_mortgage_agent_alias_arn = agents.create_agent_alias(
    existing_mortgage_agent_id, 'existing_mortgage_agent_a'
)

Store the details of the two agents that we created

In [ ]:
%store -r secret_name

secrets_helper.manage_secret("general_mortgage_questions_id", general_mortgage_questions_agent_id,secret_name)
secrets_helper.manage_secret("general_mortgage_questions_alias", general_mortgage_questions_agent_alias_id,secret_name)

secrets_helper.manage_secret("existing_mortgage_assistant_id", existing_mortgage_agent_id,secret_name)
secrets_helper.manage_secret("existing_mortgage_assistant_alias", existing_mortgage_agent_alias_id,secret_name)

In [ ]:
%store general_mortgage_questions_agent_alias_arn
%store existing_mortgage_agent_alias_arn

In the next lab, we will create an orchestrator that can route the request to the appropriate agent